In [1]:
import os
import glob
import numpy as np
from tensorflow.keras.utils import  plot_model
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from PIL import Image, ImageDraw
import pandas as pd
import seaborn as sns
from keras.callbacks import ModelCheckpoint
import keras.callbacks
import tensorflow.keras.losses
import os
import pydot
os.chdir(r'C:\Users\densh\Desktop\catdogs\keras_try')

In [2]:
#%reload_ext tensorboard

In [3]:
# Image Parameters
N_CLASSES = 2 # total number of classes
IMG_HEIGHT = 320 # the image height to be resized to
IMG_WIDTH = 320 # the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale
initial_train_data = r'C:\Users\densh\Desktop\catdogs\dataset\augmented_data'
initial_validation = r'C:\Users\densh\Desktop\catdogs\dataset\valid'
tf_train_data = r'C:\Users\densh\Desktop\catdogs\dataset\augmented_data_tf'
tf_validation = r'C:\Users\densh\Desktop\catdogs\dataset\valid_tf'

In [4]:
#uncomment to plot nn graph
os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz\bin'
os.environ["PATH"] += os.pathsep + r'C:\Users\densh\AppData\Roaming\Python\Python38\Scripts'

In [5]:
'''
#rewriting darknet dataset to ft2 folder format

with open(os.path.join(tf_validation,'train_dataset.txt'),'w') as new_txt:
    for image in glob.glob(os.path.join(initial_validation,'*.jpg')):
        title, ext = os.path.splitext(os.path.basename(image))
        txt_file_name = title + '.txt'
        cv_image = cv2.imread(image)
        cv2.imwrite(tf_train_data + '/' + title + '.jpg',cv_image)
        with open(os.path.join(initial_validation,txt_file_name),'r') as old_txt:
            labelList = old_txt.readlines()
            for label in labelList:
                label = label.strip().split()
                class_ = int(label[0])
                x_c = float(label[1])
                y_c = float(label[2])
                h   = float(label[4])
                w   = float(label[3])
        
            new_txt.write(tf_validation + '/' + title + '.jpg' + '\t' + str(int(label[0])) + '\t' + str(x_c) 
            + '\t' + str(y_c) + '\t' + str(w) + '\t' + str(h) + '\n')
'''

"\n#rewriting darknet dataset to ft2 folder format\n\nwith open(os.path.join(tf_validation,'train_dataset.txt'),'w') as new_txt:\n    for image in glob.glob(os.path.join(initial_validation,'*.jpg')):\n        title, ext = os.path.splitext(os.path.basename(image))\n        txt_file_name = title + '.txt'\n        cv_image = cv2.imread(image)\n        cv2.imwrite(tf_train_data + '/' + title + '.jpg',cv_image)\n        with open(os.path.join(initial_validation,txt_file_name),'r') as old_txt:\n            labelList = old_txt.readlines()\n            for label in labelList:\n                label = label.strip().split()\n                class_ = int(label[0])\n                x_c = float(label[1])\n                y_c = float(label[2])\n                h   = float(label[4])\n                w   = float(label[3])\n        \n            new_txt.write(tf_validation + '/' + title + '.jpg' + '\t' + str(int(label[0])) + '\t' + str(x_c) \n            + '\t' + str(y_c) + '\t' + str(w) + '\t' + str(h

In [6]:
'''
img_list = []
class_list=[]
loc_list = []
for image in glob.glob(os.path.join(tf_train_data,'*.jpg')):
    title, ext = os.path.splitext(os.path.basename(image))
    txt_file_name = title + '.txt'
    cv_image = cv2.imread(image)
    cv_image.resize(IMG_WIDTH,IMG_HEIGHT,3)
    with open(os.path.join(tf_train_data,txt_file_name),'r') as old_txt:
        labelList = old_txt.readlines()
        for label in labelList:
            label = label.strip().split()
            class_ = int(label[1])
            x_c = float(label[2])
            y_c = float(label[3])
            h     = float(label[5])
            w     = float(label[4])
    img_list.append(np.array(cv_image)/255)
    class_list.append(class_)
    loc_list.append([x_c,y_c,w,h])
img_list = np.array(img_list)
class_list =   np.array(class_list)
loc_list = np.array(loc_list)
'''

"\nimg_list = []\nclass_list=[]\nloc_list = []\nfor image in glob.glob(os.path.join(tf_train_data,'*.jpg')):\n    title, ext = os.path.splitext(os.path.basename(image))\n    txt_file_name = title + '.txt'\n    cv_image = cv2.imread(image)\n    cv_image.resize(IMG_WIDTH,IMG_HEIGHT,3)\n    with open(os.path.join(tf_train_data,txt_file_name),'r') as old_txt:\n        labelList = old_txt.readlines()\n        for label in labelList:\n            label = label.strip().split()\n            class_ = int(label[1])\n            x_c = float(label[2])\n            y_c = float(label[3])\n            h     = float(label[5])\n            w     = float(label[4])\n    img_list.append(np.array(cv_image)/255)\n    class_list.append(class_)\n    loc_list.append([x_c,y_c,w,h])\nimg_list = np.array(img_list)\nclass_list =   np.array(class_list)\nloc_list = np.array(loc_list)\n"

In [7]:
from keras.utils import to_categorical
def parse_dataset(dataset_path, ext='jpg'):
    """
    Used to extract information about our dataset. It does iterate over all images and return a DataFrame with
    the data (age, gender and sex) of all files.
    """
    def parse_info_from_file(path):
        """
        Parse information from a single file
        """
        try:
            title, ext = os.path.splitext(os.path.basename(path))
            txt_file_name = title + '.txt'
            with open(os.path.join(os.path.dirname(os.path.abspath(path)),txt_file_name),'r') as old_txt:
                labelList = old_txt.readlines()
                #print(str(labelList))
                for label in labelList:
                    label = label.strip().split()
                    class_= int(label[1])
                    x_c   = float(label[2])
                    y_c   = float(label[3])
                    w     = float(label[4])
                    h     = float(label[5])
            return class_, x_c, y_c, w, h
        except Exception as ex:
            return class_, None, None, None, None
        
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    
    records = []
    for file in files:
        info = parse_info_from_file(file)
        records.append(info)
        
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['class', 'x_c', 'y_c', 'w', 'h', 'file']
    df = df.dropna()
    
    return df

df = parse_dataset(tf_train_data)
df.head()

,class,x_c,y_c,w,h,file
0,0,0.631667,0.287500,0.153333,0.215000,C:\Users\densh\Desktop\catdogs\dataset\augment...
1,0,0.480000,0.396000,0.576000,0.372000,C:\Users\densh\Desktop\catdogs\dataset\augment...
2,0,0.616751,0.338000,0.467005,0.392000,C:\Users\densh\Desktop\catdogs\dataset\augment...
3,0,0.383249,0.662000,0.467005,0.392000,C:\Users\densh\Desktop\catdogs\dataset\augment...
4,0,0.414444,0.432907,0.588889,0.635783,C:\Users\densh\Desktop\catdogs\dataset\augment...


In [8]:
val_df = parse_dataset(tf_validation)
val_df.head()

,class,x_c,y_c,w,h,file
0,0,0.480480,0.233000,0.534535,0.294000,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
1,0,0.608000,0.277159,0.356000,0.426184,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
2,0,0.212000,0.253333,0.240000,0.389333,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
3,0,0.415916,0.336000,0.711712,0.440000,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
4,0,0.503125,0.273750,0.265000,0.367500,C:\Users\densh\Desktop\catdogs\dataset\valid_t...


In [9]:
class DataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self):
        p = np.random.permutation(len(self.df))
        train_idx = p[:]
        
        return train_idx
    
    def generate_test_split_indexes(self,length):
        p = np.random.randint(0,len(self.df),length)
        train_idx = p[:]
        
        return train_idx
    
    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        
        im = Image.open(img_path)
        im = im.resize((IMG_WIDTH, IMG_HEIGHT))
        im = np.array(im) / 255.0
        return im
        
    def generate_images(self, image_idx, is_training, batch_size):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        
        # arrays to store our batched data
        images, classes, x_c, y_c, w, h = [], [], [], [], [], []
        while True:
            for idx in image_idx:
                imag = self.df.iloc[idx]
                
                classes_ = imag['class']
                x_c_ = imag['x_c']
                y_c_ = imag['y_c']
                w_ = imag['w']
                h_ = imag['h']
                file = imag['file']
                
                im = self.preprocess_image(file)
                
                classes.append(classes_)
                x_c.append(x_c_)
                y_c.append(y_c_)
                w.append(w_)
                h.append(h_)
                images.append(im)
                #print(np.array(y_c))
                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(to_categorical(classes,2)),(np.array(x_c),np.array(y_c), np.array(w), np.array(h))]
                    images, classes, x_c, y_c, w, h = [], [], [], [], [], []
                    
            if not is_training:
                break
                
data_generator = DataGenerator(df)
train_idx = data_generator.generate_split_indexes() 
v_data_generator = DataGenerator(val_df)
valid_idx = v_data_generator.generate_split_indexes() 

In [10]:

inputs = keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, CHANNELS), name="img_input")
#classifier
x = layers.Conv2D(32, 3, activation="relu")(inputs)
x = layers.Conv2D(32, 3, activation="relu")(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(32, 3, activation="relu", padding="same")(block_1_output)
x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv2D(32, 3, activation="relu", padding="same")(block_2_output)
x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv2D(32, 3, activation="relu", padding="same")(block_3_output)
x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)
block_4_output = layers.add([x, block_3_output])

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_4_output)
block_5_output = layers.Conv2D(64, 3, activation="relu", padding="same")(x)

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_5_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_6_output = layers.add([x, block_5_output])

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_6_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_7_output = layers.add([x, block_6_output])

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_7_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_8_output = layers.add([x, block_7_output])

x = layers.Conv2D(128, 3, activation="relu", padding="same")(block_8_output)
block_9_output = layers.Conv2D(128, 3, activation="relu", padding="same")(x)

x = layers.Conv2D(128, 3, activation="relu", padding="same")(block_9_output)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
block_10_output = layers.add([x, block_9_output])

x = layers.Conv2D(128, 3, activation="relu", padding="same")(block_10_output)
block_11_output = layers.Conv2D(256, 3, activation="relu", padding="same")(x)

#x = layers.Conv2D(256, 3, activation="relu", padding="same")(block_11_output)
#x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
#block_12_output = layers.add([x, block_11_output])

#x = layers.Conv2D(256, 3, activation="relu", padding="same")(block_12_output)
#x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
#block_13_output = layers.add([x, block_12_output])

x = layers.Conv2D(128, 3, activation="relu")(block_7_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
def_set = layers.Dropout(0.2)(x)

class_output = layers.Flatten()(def_set)
class_output = layers.Dense(256, activation="relu")(class_output)
class_output = layers.Dense(128, activation="relu")(class_output )
class_output = layers.Dropout(0.2)(class_output )
class_output = layers.Dense(64, activation="relu")(class_output)
class_output = layers.BatchNormalization()(class_output)
class_output = layers.Dense(32, activation="relu")(class_output)
class_output = layers.Dense(2, activation="softmax")(class_output)
class_output = layers.Dense(2, activation="softmax",name="class_output")(class_output)
#class_output = layers.Softmax(axis=-1,name="class_output")(class_output)

#Box
Box_output = layers.Flatten()(block_11_output)
Box_output = layers.Dense(256, activation="relu")(Box_output)
Box_output = layers.Dense(128, activation="relu")(Box_output)
Box_output = layers.Dropout(0.2)(Box_output )
Box_output = layers.Dense(64, activation="relu")(Box_output)
Box_output = layers.Dropout(0.2)(Box_output )
Box_output = layers.Dense(32, activation="relu")(Box_output)
Box_output = layers.Dense(4, activation='sigmoid', name= "Box_output")(Box_output)

model2 = keras.Model(inputs, [class_output,Box_output], name="cats_dogs")


In [11]:
plot_model(model2,to_file='model.pdf',show_shapes=True)

In [12]:
init_lr = 1e-2
epochs = 50
model2.compile(
    optimizer=Adam(lr=init_lr, decay=init_lr / epochs),#tf.keras.optimizers.SGD(lr = 1e-2, momentum = 0.9),
     loss={
                  'Box_output': 'mean_squared_error',
                  'class_output': 'categorical_crossentropy'},
     loss_weights={
                  'Box_output': 1,
                  'class_output': 1},
     metrics={
        "Box_output": [
            #keras.metrics.MeanAbsolutePercentageError(),
            keras.metrics.MeanAbsoluteError()],
        "class_output": [keras.metrics.CategoricalAccuracy()],
    },
)

In [13]:
batch_size = 32
valid_batch_size = 24
train_gen =   data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = v_data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)
callbacks = [
    ModelCheckpoint(r"C:\Users\densh\Desktop\catdogs\keras_try\Checkpoint", monitor='val_loss'),
    keras.callbacks.TensorBoard(
    log_dir=r"C:\Users\densh\Desktop\catdogs\keras_try\log",write_images=True,
    histogram_freq=0,  # How often to log histogram visualizations
    embeddings_freq=0,  # How often to log embedding visualizations
    update_freq="epoch")  # How often to write logs (default: once per epoch)
]

In [15]:
#np.shape(train_gen.send(1)[1][0])

TypeError: can't send non-None value to a just-started generator

In [16]:
model2.fit(train_gen,steps_per_epoch=len(train_idx)//(batch_size), callbacks=callbacks, epochs=epochs, validation_data=valid_gen, validation_steps=len(valid_idx)//valid_batch_size)

Epoch 1/50
 60/186 [========>.....................] - ETA: 53:07 - loss: 0.8526 - class_output_loss: 0.6089 - Box_output_loss: 0.2437 - class_output_categorical_accuracy: 0.7067 - Box_output_mean_absolute_error: 0.4668

KeyboardInterrupt: 

In [17]:
model2.save(
    r"C:\Users\densh\Desktop\catdogs\keras_try\models",
    overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True)


INFO:tensorflow:Assets written to: C:\Users\densh\Desktop\catdogs\keras_try\models\assets


In [17]:
train_idx = v_data_generator.generate_test_split_indexes(10)
test_gen =   v_data_generator.generate_images(train_idx, is_training=False, batch_size=1)
c = model2.predict(test_gen,steps=len(train_idx))


In [18]:
c

[array([[0.28716555, 0.7128345 ],
        [0.28716555, 0.7128345 ],
        [0.28716555, 0.7128345 ],
        [0.28716555, 0.7128345 ],
        [0.28716555, 0.7128345 ],
        [0.28716555, 0.7128345 ],
        [0.28716555, 0.7128345 ],
        [0.28716555, 0.7128345 ],
        [0.28716555, 0.7128345 ],
        [0.28716555, 0.7128345 ]], dtype=float32),
 array([[0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.]], dtype=float32)]

In [19]:
val_df.iloc[train_idx,]

,class,x_c,y_c,w,h,file
307,0,0.590000,0.445333,0.172000,0.261333,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
146,1,0.537538,0.408000,0.828829,0.496000,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
171,1,0.421922,0.512000,0.501502,0.208000,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
96,1,0.470000,0.502667,0.364000,0.418667,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
269,0,0.484496,0.268000,0.589147,0.268000,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
378,1,0.577000,0.465333,0.366000,0.381333,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
123,0,0.536585,0.350000,0.650407,0.550000,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
341,1,0.442000,0.371212,0.712000,0.737374,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
283,1,0.450739,0.240000,0.625616,0.473333,C:\Users\densh\Desktop\catdogs\dataset\valid_t...
262,1,0.552000,0.454955,0.500000,0.903904,C:\Users\densh\Desktop\catdogs\dataset\valid_t...


In [5]:
pip install cuDNN 

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement cuDNN (from versions: none)
ERROR: No matching distribution found for cuDNN


In [9]:
import tensorflow as tf
gtf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]